In [1]:
import pandas as pd
import numpy as np
from terminaltables import AsciiTable

C:\Users\Deutsch\AppData\Local\Temp\ipykernel_6792\2243802572.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
#SE IMPORTA EL CSV
NIO_data = pd.read_csv('../CSV/NIO2.csv')
#SE BORRAN LAS COLUMNAS QUE NO SE USARAN
NIO_data = NIO_data.drop(columns=['Open', 'High', 'Low', 'Close','Volume'])
#SE CALCULA EL CAMBIO PORCENTUAL DE CADA REGISTRO DE ACUERDO AL REGISTRO ANTERIOR
percentage = [0,]
for i in range(1, len(NIO_data)):
    percent = (NIO_data['Adj Close'][i] - NIO_data['Adj Close'][i-1])/NIO_data['Adj Close'][i-1]
    percentage.append(percent)
NIO_data.insert(2, 'Change %', percentage)

#SE CALCULA LA DESVIACION ESTANDAR DE LOS 20 DATOS ANTERIORES
std_dev = [0,]*20
for i in range(21, len(percentage) + 1):
    standar_dev = np.std(percentage[i-20:i])
    std_dev.append(standar_dev)
NIO_data.insert(3, 'std dev', std_dev)

#SE ASIGNA EL ESTADO DE ACUERDO A LA DESVIACION ESTANDAR DE LAS 20 SEMANAS ANTERIORES
states = [0,]*20
for j in range(20, len(percentage)):
    change = percentage[j]
    desv = std_dev[j]
    if change <= -2*desv:
        state = "B3"
    elif -2*desv < change <= -desv:
        state = "B2"
    elif -desv < change < 0:
        state = "B1"
    elif 0 <= change < desv:
        state = "S1"
    elif desv <= change < 2*desv:
        state = "S2"
    elif 2*desv <= change:  
        state = "S3"
    states.append(state)
NIO_data.insert(4, 'Current State', states)

#SE ASIGNA UNA NUEVA COLUMNA CON EL ESTADO SIGUIENTE AL REGISTRO
next_state_column=[]
for i in range(len(NIO_data)-1):
    next_state_column.append(NIO_data['Current State'][i+1])

previows_state_column=[0]
for i in range(1,len(NIO_data)-1):
    previows_state_column.append(NIO_data['Current State'][i-1])
    
#SE BORRA LA ULTIMA FILA POR QUE NO TIENE ESTADO SIGUIENTE
NIO_data = NIO_data.drop(NIO_data.index[-1])
NIO_data.insert(5, 'Next State', next_state_column)
NIO_data.insert(4, 'Previous State', previows_state_column)

In [3]:
def calculo_inverso(data):
    def desviacion_estandar(data):
        std_dev = []
        for i in range(len(data)):
            if i < 20:
                std_dev.append(0)
            else:
                standar_dev = np.std(data['Change %'][i-20:i+1])
                std_dev.append(standar_dev)
        return std_dev
    
    def calculo_cambios_inversos(data):
        cambio_inverso = []
        precios_estimados = []
        for i in range(len(data)):
            estado_actual = data['Current State'][i]
            precio_anterior = data['Adj Close'][i]
            cambio_previo = data['Change %'][i]
            std_dev = data['std dev'][i]
            cambio = 0
            if estado_actual == "B3":
                cambio = -2 * std_dev
            elif estado_actual == "B2":
                cambio = (-std_dev + cambio_previo) / 2
            elif estado_actual == "B1":
                cambio = cambio_previo / 2
            elif estado_actual == "S1":
                cambio = cambio_previo / 2
            elif estado_actual == "S2":
                cambio = (std_dev + cambio_previo) / 2
            elif estado_actual == "S3":
                cambio = 2 * std_dev
            cambio_inverso.append(cambio)
            precio_estimado = precio_anterior * (1 + cambio)
            precios_estimados.append(precio_estimado)
        return cambio_inverso, precios_estimados
    
    data['std dev'] = desviacion_estandar(data)
    cambio_inverso, precios_estimados = calculo_cambios_inversos(data)

    data['Inverse Change %'] = cambio_inverso
    data['Estimated Price'] = precios_estimados

    return data

NIO_data_inverse = calculo_inverso(NIO_data)

complete_table_inverse = AsciiTable([NIO_data_inverse.columns]+list(NIO_data_inverse.values))
print("Complete Table \n",complete_table_inverse.table)

Complete Table 
 +------------+-----------+-------------------------+----------------------+----------------+---------------+------------+-------------------------+--------------------+
| Date       | Adj Close | Change %                | std dev              | Previous State | Current State | Next State | Inverse Change %        | Estimated Price    |
+------------+-----------+-------------------------+----------------------+----------------+---------------+------------+-------------------------+--------------------+
| 2020-04-01 | 2.65      | 0.0                     | 0.0                  | 0              | 0             | 0          | 0.0                     | 2.65               |
| 2020-04-02 | 2.39      | -0.09811320754716973    | 0.0                  | 0              | 0             | 0          | 0.0                     | 2.39               |
| 2020-04-03 | 2.4       | 0.004184100418409952    | 0.0                  | 0              | 0             | 0          | 0.0             